In [1]:
#Running python 3.8.12

import pychrono as chrono
from pychrono import irrlicht as chronoirr
import numpy as np
import math as m
import sys
import pandas as pd

chrono.SetChronoDataPath('data')
meshpath = 'C:/Users/sbrege/Documents/coding/mesh/'

In [2]:
#Code for collision reports
class RatReportContactCallback(chrono.ReportContactCallback):
     def __init__(self):
          chrono.ReportContactCallback.__init__(self)
          self.bodyA = None
          self.bodyB = None
          self.nameA = None
          self.nameB = None
          self.collideCount = 0
          self.wallCollide = False

     def OnReportContact(self,vA,vB,cA,dist,rad,force,torque,modA,modB):
          self.bodyA = chrono.CastContactableToChBody(modA)
          self.nameA = self.bodyA.GetName()
          self.bodyB = chrono.CastContactableToChBody(modB)
          self.nameB = self.bodyB.GetName() #always seems to be the rat but may not always be true

          if self.nameA == 'wall':
               self.collideCount+=1

          if self.collideCount == 10:
               self.wallCollide = True
               print('wall collide')
               self.collideCount = 0

          #print ('contact: point A=' , vA.z, 'first body:', nameA, 'second body:', nameB)

          return True        # return False to stop reporting contacts



In [3]:
#Useful quaternions
# z2x = chrono.ChQuaternionD()
# z2x.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))
# z2y = chrono.ChQuaternionD()
# z2y.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))

# #Turns chrono vector into a numpy array
# def vec2Array(vector):
#     return np.asarray([vector.x, vector.y, vector.z])

# def array2Vec(array):
#     return chrono.ChVectorD(array[0], array[1], array[2])

# #Class that contains the chrono system
# class ratSim():
#     #init, duh
#     def __init__(self, arena='kex', rat_type='sphere', p_len=0.55, p_wid=0.15, p_height=0.15, a_len=4, a_wid=2, start_pos=[0,2], goals=[[2,2],[2,1],[3,1],[3,0],[4,0]], reportContact=False):

#         #Initialising chrono system
#         self.system = chrono.ChSystemNSC()
#         self.material = chrono.ChMaterialSurfaceNSC()

#         chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
#         chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)

#         #arena parameters

#         self.arena = arena
#         self.rat_type = rat_type
#         self.p_len=p_len
#         self.p_wid=p_wid
#         self.p_height=p_height
#         self.gamma = (p_len-p_wid)
#         self.a_len = a_len
#         self.a_wid = a_wid
#         self.start_pos = np.asarray(start_pos)
#         #goals [left/right,down/up] [-1/1, -1/1]
#         self.goals = np.asarray(goals)
#         self.reportContact = reportContact

#         #Setups the system for simulation
#         self.reset()

#     def makeKexArena(self, horizontal, vertical):
#         pieces = []
#         for x in range(horizontal):
#             for y in range(vertical):
#                 #Vertical
#                 piece = chrono.ChBodyEasyBox(self.p_len, self.p_height, self.p_wid, 3000, True, True, self.material)
#                 piece.SetPos(chrono.ChVectorD(self.gamma*x + self.gamma/2, -0.2, self.gamma*y))
#                 piece.SetBodyFixed(True)
#                 piece.SetName('KexPieceV-' + str(x) + '-' + str(y))
#                 pieces.append(piece)

#                 #Horizontal
#                 piece = chrono.ChBodyEasyBox(self.p_wid, self.p_height, self.p_len, 3000, True, True, self.material)
#                 piece.SetPos(chrono.ChVectorD(self.gamma*x, -0.2, self.gamma*y + self.gamma/2))
#                 piece.SetBodyFixed(True)
#                 piece.SetName('KexPieceH-' + str(x) + '-' + str(y))
#                 pieces.append(piece)

#                 #End pieces

#                 if y == vertical-1:
#                     #Vertical
#                     piece = chrono.ChBodyEasyBox(self.p_len, self.p_height, self.p_wid, 3000, True, True, self.material)
#                     piece.SetPos(chrono.ChVectorD(self.gamma*(x) + self.gamma/2, -0.2, self.gamma*(y+1)))
#                     piece.SetBodyFixed(True)
#                     piece.SetName('KexPieceV-' + str(x+1) + '-' + str(y))
#                     pieces.append(piece)

#                 if x == horizontal-1:
#                     #Horizontal
#                     piece = chrono.ChBodyEasyBox(self.p_wid, self.p_height, self.p_len, 3000, True, True, self.material)
#                     piece.SetPos(chrono.ChVectorD(self.gamma*(x+1), -0.2, self.gamma*(y) + self.gamma/2))
#                     piece.SetBodyFixed(True)
#                     piece.SetName('KexPieceH-' + str(x) + '-' + str(y+1))
#                     pieces.append(piece)

#         #Add pieces to system
#         for piece in pieces:
#             self.system.Add(piece)

#     #Clears and adds elements to system
#     def reset(self):
#         #Clears system, duh
#         self.system.Clear()
#         self.system.Set_G_acc(chrono.ChVectorD(0,0,0))

#         if self.arena == 'kex':

#             self.makeKexArena(self.a_len, self.a_wid)
        
#         elif self.arena == 'box':
#             floor = chrono.ChBodyEasyBox(5, self.p_height, 5, 3000, True, True, self.material)
#             floor.SetPos(chrono.ChVectorD(0, -0.2, 0))
#             floor.SetBodyFixed(True)
#             self.system.Add(floor)

#         #rat mesh
#         if self.rat_type == 'sphere':
#             self.rat = chrono.ChBodyEasySphere(0.02, 3000, True, True, self.material)
#         self.rat.SetBodyFixed(False)
#         self.rat.SetCollide(False)
#         self.rat.SetName('rat')
#         #self.rat.AddAsset(chrono.ChColorAsset(1,0.4,0.4))
#         self.rat.SetPos(chrono.ChVectorD(self.start_pos[0]*self.gamma, -0.1, self.start_pos[1]*self.gamma))
        
#         self.system.Add(self.rat)

#         #Head to show rat direction
#         self.head = chrono.ChBodyEasyBox(0.02, 0.008, 0.008, 3000, True, True, self.material)
#         self.head.SetBodyFixed(False)
#         self.head.SetCollide(False)
#         self.head.SetPos(chrono.ChVectorD(self.start_pos[0]*self.gamma + 0.05, -0.1, self.start_pos[1]*self.gamma))
#         self.head.AddAsset(chrono.ChColorAsset(1,0.4,0.4))
#         self.system.Add(self.head)

#         #Link for head and rat
#         rLink = chrono.ChLinkMateFix()
#         rLink.Initialize(self.head, self.rat)
#         self.system.Add(rLink)

#         # right = chrono.ChQuaternionD()
#         # right.Q_from_AngAxis(chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))
#         # self.rat.SetRot(right)
#         # rot = self.rat.GetRot()
#         # print(rot)
#         # ax = self.rat.GetRotAxis()
#         # print(ax)
#         # ang = self.rat.GetRotAngle()
#         # print(ang)

#         #Joint for actuator and rat

#         self.joint = chrono.ChBodyEasyBox(0.05,0.05,0.05, 3000, False, False, self.material)
#         #actual position unimportant, can be ignored
#         self.joint.SetPos(chrono.ChVectorD(0, -0.1, 0.6))
#         self.joint.SetName('joint')
#         self.joint.SetBodyFixed(True)
#         self.system.Add(self.joint)

#         #actuator to move rat

#         self.actuator = chrono.ChLinkLockLock()
#         self.actuator.Initialize(self.rat, self.joint, chrono.CSYSNORM)
#         self.system.Add(self.actuator)

#         self.actuator.SetMotion_X(chrono.ChFunction_Const(0))
#         self.actuator.SetMotion_Z(chrono.ChFunction_Const(0))

#         if self.reportContact:
#             self.rep = RatReportContactCallback()

#     #Sets rat up for next move
#     def move(self):
#         return

#     #Runs IRR simulation, which is visualized
#     def runIRR(self):
#         #Boring setup, self explanatory
#         self.application = chronoirr.ChIrrApp(self.system, "Grid cell sim", chronoirr.dimension2du(1224, 900), False)
#         self.application.SetTryRealtime(True)

#         self.application.AddTypicalSky()
#         self.application.AddTypicalLights()
#         self.application.AddTypicalCamera(chronoirr.vector3df(2, 1, -0.5))
#         self.application.AddLightWithShadow(chronoirr.vector3df(20.0, 35.0, -25.0), chronoirr.vector3df(0, 0, 0), 55, 20, 55, 35, 512,
#                                     chronoirr.SColorf(0.6, 0.8, 1.0))

#         self.application.AssetBindAll()
#         self.application.AssetUpdateAll()
#         self.application.AddShadowAll()

#         self.solver = chrono.ChSolverPSOR()
#         self.solver.SetMaxIterations(50)
#         self.system.SetSolver(self.solver)

#         #Timestep 
#         dt = 0.001
#         self.application.SetTimestep(dt)
#         self.application.SetTryRealtime(False)

#         #Timestep counter
#         ts = 0
#         timeout = 0

#         #Index counter for target list
#         tIndex = 0

#         #previous action in sequence
#         pMove = np.asarray([0,0])

#         #contains all previous actions for movement
#         tCumulative = np.asarray([0,0])

#         nextTarget = True

#         #Behavioral recordings
#         vel_recordings = []

#         #Constant determining new vector distance from current
#         resolution = 1

#         #Constant determing how far it is allowed to turn in one timestep in radians
#         turnRate = m.pi/300

#         #Quaternion for rotating rat
#         rotator = chrono.ChQuaternionD()
#         #self.actuator.SetMotion_axis(chrono.ChVectorD(0, 0, 0))

#         bruh = 0

#         #Runs the IRR sim one timestep at a time, simulation will stop when pop-up is closed or application is stopped inside the loop
#         while self.application.GetDevice().run() :
#             self.application.BeginScene(True, True, chronoirr.SColor(255, 140, 161, 192))

#             self.application.DrawAll()

#             t = self.system.GetChTime()

#             if self.reportContact:
#                 self.system.GetContactContainer().ReportAllContacts(self.rep)

#             if tIndex < len(self.goals):
#                 linearVel = vec2Array(self.rat.GetPos_dt())
#                 ratPos = np.asarray([self.rat.GetPos().x, self.rat.GetPos().z])
#                 posVector = ratPos - self.start_pos*self.gamma
#                 #print(posVector)
#                 vel_recordings.append(linearVel)
#                 target = np.asarray(self.goals[tIndex])

#                 #Vector of position to target
#                 targetVector = target - (ratPos/self.gamma)

#                 #Angle to target in radians
#                 targetAngle = m.atan(targetVector[1]/targetVector[0])

#                 angleSign = np.sign(targetAngle)
#                 #print(targetAngle)
#                 #print(targetVector)
#                 #Want to use smallest magnitude angle adjustment
#                 if turnRate < np.linalg.norm(targetAngle):
#                     theta = turnRate*angleSign

#                 else:
#                     theta = targetAngle
#                     #print(targetAngle)
#                     #print(turnRate*angleSign)

#                     #Rotate rat in direction of target
#                     #rotator.Q_from_AngAxis(targetAngle, chrono.ChVectorD(0, 1, 0))
#                     #self.rat.SetRot(rotator)

#                     #Movement
#                     #self.actuator.SetMotion_X(chrono.ChFunction_Const((resolution * m.cos(turnRate*angleSign))*self.gamma + posVector[0]))
#                     #self.actuator.SetMotion_Z(chrono.ChFunction_Const((resolution * m.sin(turnRate*angleSign))*self.gamma + posVector[1]))

#                 #Creating coordinate space for vector rotation
#                 cSpace = np.asarray([[m.cos(theta), -m.sin(theta)],[m.sin(theta), m.cos(theta)]])

#                 #Rotating vector
#                 newV = np.dot(cSpace,oldV)

#                 oldV = newV


#                 # xDist = (resolution * m.cos(theta))*self.gamma + posVector[0]
#                 # zDist = (resolution * m.sin(theta))*self.gamma + posVector[1]

#                 #     if bruh == 10:
#                 #         print("Saggy Nuts")
#                 #         print('xDist', xDist)
#                 #         print('zDist', zDist)
#                 #         print('PosVector', posVector)
#                 #         print('targetAngle', targetAngle)
#                 #         print('target', target)
#                 #         print('targetVector', targetVector)
#                 #         print('actual movement X', xDist-posVector[0])
#                 #         print('actual movement Z', zDist-posVector[1])
#                 #         bruh = 0
#                 #     bruh+=1
#                 #     self.actuator.SetMotion_X(chrono.ChFunction_Const(xDist))
#                 #     self.actuator.SetMotion_Z(chrono.ChFunction_Const(zDist))

#                 # else:
#                 #     #rotator.Q_from_AngAxis(turnRate*np.sign(targetAngle) , chrono.ChVectorD(0, 1, 0))
#                 #     #self.rat.SetRot(rotator)
#                 #     #print(targetAngle)
#                 #     xDist = (resolution * m.cos(targetAngle))*self.gamma + posVector[0]
#                 #     #print(xDist)
#                 #     zDist = (resolution * m.sin(targetAngle))*self.gamma + posVector[1]
#                 #     #print(zDist)

#                 #     #Movement
#                 #     self.actuator.SetMotion_X(chrono.ChFunction_Const(xDist))
#                 #     self.actuator.SetMotion_Z(chrono.ChFunction_Const(zDist))

#                 if np.sum(np.abs(targetVector)) < 0.2:
#                     print('SHITSHITSHITHSIT')

#                     #rotator.Q_from_AngAxis(chrono.CH_C_PI / 2, chrono.ChVectorD(0, 1, 0))
#                     #self.rat.SetRot(rotator)
#                     #self.actuator.SetMotion_axis(chrono.ChVectorD(0, 0, 0))

#                     #Horizontal
#                     #self.actuator.SetMotion_X(chrono.ChFunction_Const((target[0]-self.start_pos[0])*self.gamma))

#                     #Vertical
#                     #self.actuator.SetMotion_Z(chrono.ChFunction_Const((target[1]-self.start_pos[1])*self.gamma))

#                     #self.actuator.SetMotion_axis(chrono.ChVectorD(0, 0, 0))
#                     #self.actuator.SetMotion_ang(chrono.ChFunction_Const(targetAngle))
                    

#                     tCumulative += target
#                     nextTarget = False
#                     timeout = 0
#                     tIndex += 1

#                 self.application.DoStep()

#                 ts+=1
#                 timeout += 1

#                 #Determines when it has finished a move
#                 # if timeout >= 0.07/dt:
#                 #     if np.sum(np.abs(linearVel)) < 0.2:
#                 #         nextTarget = True
#                 #         tIndex += 1
#                 #         timeout = 0
            
#             self.application.EndScene()
#         #Closes down IRR application fully and resets so it can be run again.
#         self.application.GetDevice().closeDevice()
#         self.reset()
#         #print(ratPos/self.gamma)
#         return vel_recordings

        


In [4]:
#Useful quaternions
z2x = chrono.ChQuaternionD()
z2x.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))
z2y = chrono.ChQuaternionD()
z2y.Q_from_AngAxis(-chrono.CH_C_PI / 2 , chrono.ChVectorD(0, 1, 0))

#Turns chrono vector into a numpy array
def vec2Array(vector):
    return np.asarray([vector.x, vector.y, vector.z])

def array2Vec(array):
    return chrono.ChVectorD(array[0], array[1], array[2])

#Helper functions for calculating rat movement

#Calculates angle to rotate vector, with b being the goal point and a the vector to rotate
def calc_angle(a, b):
    sign = np.sign(b[1])
    return sign*m.acos((a@b.T)/(np.linalg.norm(a) * np.linalg.norm(b) + 0.001))

def calc_angle2(a, b):
    #Equation taken from: https://www.mathworks.com/matlabcentral/answers/180131-how-can-i-find-the-angle-between-two-vectors-including-directional-information
    subtract = a[0]*b[1] - a[1]*b[0]
    add = a[0]*b[0] + a[1]*b[1]
    return m.atan2(subtract, add)

#Rotates vector V by theta radians
def rotate_vec(V, theta):
    #creating helper coord-space for rotation
    coord_space = np.array([[m.cos(theta),-m.sin(theta)], [m.sin(theta),m.cos(theta)]])
    return (coord_space@V.T).T

#Class that contains the chrono system
class ratSim():
    """
    Creates simulation of spatial cognition experimental maze for generating behavioral data, testing models, whatever you want to use it for.
    """
    #init, duh
    def __init__(self, arena='kex', rat_type='sphere', p_params=[0.55, 0.15, 0.15], a_params=[4, 2], goals=[[0,2],[2,2],[2,1],[3,1],[3,0],[4,0]], theta_c=0.5, dt=0.001, vec_mag=0.8, acc_mag=0, t_radius=0.2, report_contact=False):
        """
        Initialize rat simulation.

        :param str arena: What arena to simulate, currently only supports kex
        :param str rat_type: Type of object to act as rat/subject, currently only supports sphere
        :param list p_params: Dimensions of each arena piece, from birds eye, [horizontal, depth, vertical]
        :param list a_params: Dimensions of total arena, [horizontal, vertical]
        :param list2d goals: Arena coords of goals, with rat starting at first point. From birds eye Bottom Left [0,0], Top Right [a_params[0], a_params[1]]
        :param float theta_c: Theta constant, determines max angle rat can turn (larger = smaller angle)
        :param float dt: Timestep, smaller = more precise, but longer. Also adjusts max angle
        :param float vec_mag: Affects rat speed, larger = faster
        :param float acc_mag: How much rat accelerates when travelling in straight line, set to 0 for no acceleration.
        :param float t_radius: Controls switching to new target
        :param bool report_contact: Whether to use callback report, keep False if you don't understand it.
        """


        #Initialising chrono system
        self.system = chrono.ChSystemNSC()
        self.material = chrono.ChMaterialSurfaceNSC()

        chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
        chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)

        #arena parameters

        self.arena = arena
        self.rat_type = rat_type
        self.p_len = p_params[0]
        self.p_wid = p_params[1]
        self.p_height = p_params[2]
        self.a_len = a_params[0]
        self.a_wid = a_params[1]
        self.start_pos = np.asarray(goals[0])

        #goals [left/right,down/up] [-1/1, -1/1]
        self.goals = np.asarray(goals[1:])
        self.report_contact = report_contact

        self.theta_c = theta_c
        self.dt = dt
        self.vec_mag = vec_mag
        self.acc_mag = acc_mag
        self.t_radius = t_radius

        #Setups the system for simulation
        self.reset()

    def makeKexArena(self, horizontal, vertical):
        pieces = []
        for x in range(horizontal):
            for y in range(vertical):
                #Vertical
                piece = chrono.ChBodyEasyBox(self.p_len, self.p_height, self.p_wid, 3000, True, True, self.material)
                piece.SetPos(chrono.ChVectorD(self.gamma*x + self.gamma/2, -0.2, self.gamma*y))
                piece.SetBodyFixed(True)
                piece.SetName('KexPieceV-' + str(x) + '-' + str(y))
                pieces.append(piece)

                #Horizontal
                piece = chrono.ChBodyEasyBox(self.p_wid, self.p_height, self.p_len, 3000, True, True, self.material)
                piece.SetPos(chrono.ChVectorD(self.gamma*x, -0.2, self.gamma*y + self.gamma/2))
                piece.SetBodyFixed(True)
                piece.SetName('KexPieceH-' + str(x) + '-' + str(y))
                pieces.append(piece)

                #End pieces

                if y == vertical-1:
                    #Vertical
                    piece = chrono.ChBodyEasyBox(self.p_len, self.p_height, self.p_wid, 3000, True, True, self.material)
                    piece.SetPos(chrono.ChVectorD(self.gamma*(x) + self.gamma/2, -0.2, self.gamma*(y+1)))
                    piece.SetBodyFixed(True)
                    piece.SetName('KexPieceV-' + str(x+1) + '-' + str(y))
                    pieces.append(piece)

                if x == horizontal-1:
                    #Horizontal
                    piece = chrono.ChBodyEasyBox(self.p_wid, self.p_height, self.p_len, 3000, True, True, self.material)
                    piece.SetPos(chrono.ChVectorD(self.gamma*(x+1), -0.2, self.gamma*(y) + self.gamma/2))
                    piece.SetBodyFixed(True)
                    piece.SetName('KexPieceH-' + str(x) + '-' + str(y+1))
                    pieces.append(piece)

        #Add pieces to system
        for piece in pieces:
            self.system.Add(piece)

    #Clears and adds elements to system
    def reset(self):
        #Clears system, duh
        self.system.Clear()

        #Don't need gravity
        self.system.Set_G_acc(chrono.ChVectorD(0,0,0))

        #Constant for adjusting arena coordinate space to actual coordinate space
        self.gamma = (self.p_len-self.p_wid)
        #Max angle that rat can turn in one step, adjusted by timestep and theta_c (parameter)
        self.theta_max = (self.dt*m.pi)/self.theta_c
        #self.theta_max = (m.pi/8)

        if self.arena == 'kex':

            self.makeKexArena(self.a_len, self.a_wid)
        
        elif self.arena == 'box':
            floor = chrono.ChBodyEasyBox(5, self.p_height, 5, 3000, True, True, self.material)
            floor.SetPos(chrono.ChVectorD(0, -0.2, 0))
            floor.SetBodyFixed(True)
            self.system.Add(floor)

        #rat mesh
        if self.rat_type == 'sphere':
            self.rat = chrono.ChBodyEasySphere(0.015, 3000, True, True, self.material)
        self.rat.SetBodyFixed(False)
        self.rat.SetCollide(False)
        self.rat.SetName('rat')
        self.rat.AddAsset(chrono.ChColorAsset(0,0,0))
        self.rat.SetPos(chrono.ChVectorD(self.start_pos[0]*self.gamma, -0.1, self.start_pos[1]*self.gamma))
        
        self.system.Add(self.rat)

        #Head to show rat direction
        # self.head = chrono.ChBodyEasyBox(0.05, 0.008, 0.008, 3000, True, True, self.material)
        # self.head.SetBodyFixed(False)
        # self.head.SetCollide(False)
        # self.head.SetPos(chrono.ChVectorD(self.start_pos[0]*self.gamma, -0.1, self.start_pos[1]*self.gamma))
        # self.head.AddAsset(chrono.ChColorAsset(1,0.4,0.4))
        # self.system.Add(self.head)

        # #Link for head and rat
        # rLink = chrono.ChLinkMateFix()
        # rLink.Initialize(self.head, self.rat)
        # self.system.Add(rLink)

        #Joint for actuator and rat

        self.joint = chrono.ChBodyEasyBox(0.05,0.05,0.05, 3000, False, False, self.material)
        #actual position unimportant, can be ignored
        self.joint.SetPos(chrono.ChVectorD(0, -0.1, 0.6))
        self.joint.SetName('joint')
        self.joint.SetBodyFixed(True)
        self.system.Add(self.joint)

        #actuator to move rat

        self.actuator = chrono.ChLinkLockLock()
        self.actuator.Initialize(self.rat, self.joint, chrono.CSYSNORM)
        self.system.Add(self.actuator)

        self.actuator.SetMotion_X(chrono.ChFunction_Const(0))
        self.actuator.SetMotion_Z(chrono.ChFunction_Const(0))

        if self.report_contact:
            self.rep = RatReportContactCallback()

    #Determines what theta to use
    def det_angle(self, theta):
        if np.abs(theta) <= self.theta_max:
            return theta
        else:
            return np.sign(theta) * self.theta_max

    #Determines whether rat still on arena or if floating
    def detDanger(self, p_vector):
        adjWidth = self.p_wid/2

        smaller = np.asarray([-self.p_wid, -self.p_wid])
        larger = np.asarray([self.p_wid + self.a_len*self.gamma, self.p_wid + self.a_wid*self.gamma])

        if (np.any(p_vector < smaller) or np.any(p_vector > larger)):
            return True

        for x in range(self.a_len):
            for z in range(self.a_wid):
                smaller = np.asarray([x*self.gamma + adjWidth, z*self.gamma + adjWidth])
                larger = np.asarray([(x+1)*self.gamma - adjWidth, (z+1)*self.gamma - adjWidth])
                if (np.all(p_vector > smaller) and np.all(p_vector < larger)):
                    return True
        else:
            return False

    def runStep(self, g_ind, m_vector, p_vector, g_vector, vel_recordings, running):
        #Only moves rat whils there are still goals to move to
        if  running:
            t_vector = g_vector - p_vector

            theta = self.det_angle(calc_angle2(m_vector, t_vector))

            m_vector = rotate_vec(m_vector, theta)
            
            #running theta value, adjusts until it is large enough to escape
            r_theta = 0

            i = 0

            #If run off edge; don't
            while self.detDanger(m_vector*self.dt + p_vector):
                # adj_vector = self.goals[g_ind-1]*self.gamma - p_vector

                # r_theta += self.det_angle(calc_angle2(m_vector, adj_vector))

                r_theta += 0.05

                #Whichever direction needs smallest turn, choose that
                if i %2 == 0:
                    m_vector = rotate_vec(m_vector, r_theta)
                else:
                    m_vector = rotate_vec(m_vector, -r_theta)

            p_vector = m_vector*self.dt + p_vector
            
            #moves rat
            self.rat.SetPos(chrono.ChVectorD(p_vector[0], -0.1, p_vector[1]))
            #self.head.SetPos(chrono.ChVectorD(p_vector[0], -0.1, p_vector[1]))
            #self.head.SetRot()

            #Records linear velocity
            vel_recordings.append(m_vector)


            #If goal is the last one, must fully reach it.
            if g_ind+1 == len(self.goals):
                radius = 0.01
            else:
                radius = self.t_radius
        
            if np.sum(np.abs(t_vector)) < radius:
                print('Goal reached, moving on')
                g_ind += 1
                if g_ind < len(self.goals):
                    g_vector = self.goals[g_ind]*self.gamma
                else:
                    running = False
        return g_ind, m_vector, p_vector, g_vector, vel_recordings, running

    #Runs IRR simulation, which is visualized. Used to test whether things are working and it looks cool
    def runIRR(self):

        #Boring standard simulation setup
        self.application = chronoirr.ChIrrApp(self.system, "Kex sim", chronoirr.dimension2du(1224, 900), False)
        self.application.SetTryRealtime(True)

        self.application.AddTypicalSky()
        self.application.AddTypicalLights()
        self.application.AddTypicalCamera(chronoirr.vector3df(2, 1, -0.5))
        self.application.AddLightWithShadow(chronoirr.vector3df(20.0, 35.0, -25.0), chronoirr.vector3df(0, 0, 0), 55, 20, 55, 35, 512,
                                    chronoirr.SColorf(0.6, 0.8, 1.0))

        self.application.AssetBindAll()
        self.application.AssetUpdateAll()
        self.application.AddShadowAll()

        self.solver = chrono.ChSolverPSOR()
        self.solver.SetMaxIterations(50)
        self.system.SetSolver(self.solver)

        #Timestep 
        self.application.SetTimestep(self.dt)
        self.application.SetTryRealtime(False)

        #Behavioral recordings
        vel_recordings = []

        #Initialization for movement calculations

        #initializes movement vector to rotate towards target
        m_vector = np.array([self.vec_mag,0]) 
        #Rat position in actual coordinates
        p_vector = self.start_pos * self.gamma
        #Goal position in actual coordinates
        g_ind = 0
        g_vector = self.goals[g_ind] * self.gamma
        #Vector from position to goal
        t_vector = g_vector - p_vector
        
        #Rotates initial vector to face first target
        theta = calc_angle(m_vector, t_vector)
        m_vector = rotate_vec(m_vector, theta)

        #Stops system if something goes wrong preventing need for kernel restart
        fuckedUp = False

        #Runs the IRR sim one timestep at a time, simulation will stop when pop-up is closed or application is stopped inside the loop
        while self.application.GetDevice().run():
            self.application.BeginScene(True, True, chronoirr.SColor(255, 140, 161, 192))

            self.application.DrawAll()

            t = self.system.GetChTime()

            if self.report_contact:
                self.system.GetContactContainer().ReportAllContacts(self.rep)

            #Only moves rat whils there are still goals to move to
            if  g_ind < len(self.goals) and not fuckedUp:

                t_vector = g_vector - p_vector

                theta = self.det_angle(calc_angle2(m_vector, t_vector))

                m_vector = rotate_vec(m_vector, theta)
                
                #running theta value, adjusts until it is large enough to escape
                r_theta = 0

                i = 0

                #If run off edge; don't
                while self.detDanger(m_vector*self.dt + p_vector) and not fuckedUp:
                    adj_vector = self.goals[g_ind-1]*self.gamma - p_vector

                    r_theta += self.det_angle(calc_angle2(m_vector, adj_vector))

                    if i %2 == 0:
                        m_vector = rotate_vec(m_vector, r_theta)
                    else:
                        m_vector = rotate_vec(m_vector, -r_theta)

                    i += 1
                    if i == 100:
                        fuckedUp = True

                p_vector = m_vector*self.dt + p_vector
                
                #moves rat
                self.rat.SetPos(chrono.ChVectorD(p_vector[0], -0.1, p_vector[1]))
                #self.head.SetPos(chrono.ChVectorD(p_vector[0], -0.1, p_vector[1]))
                #self.head.SetRot()

                #Records linear velocity
                vel_recordings.append(m_vector)


                #If goal is the last one, must fully reach it.
                if g_ind+1 == len(self.goals):
                    radius = 0.01
                else:
                    radius = self.t_radius
            
                if np.sum(np.abs(t_vector)) < radius:
                    print('Goal reached, moving on')
                    g_ind += 1
                    if g_ind < len(self.goals):
                        g_vector = self.goals[g_ind]*self.gamma

                self.application.DoStep()
            
            self.application.EndScene()
        #Closes down IRR application fully and resets so it can be run again.
        self.application.GetDevice().closeDevice()
        self.reset()
        #print(ratPos/self.gamma)

        if fuckedUp:
            print('Something went wrong! Try adjusting parameters')
            return None
        else:
            return vel_recordings

    #Runs simulation without visualization
    def run(self):
        self.system.SetChTime(0)
        t = self.system.GetChTime()

        self.solver = chrono.ChSolverPSOR()
        self.solver.SetMaxIterations(50)
        self.system.SetSolver(self.solver)

        #Behavioral recordings
        vel_recordings = []

        #Initialization for movement calculations

        #initializes movement vector to rotate towards target
        m_vector = np.array([self.vec_mag,0]) * 1
        #Rat position in actual coordinates
        p_vector = self.start_pos * self.gamma
        #Goal position in actual coordinates
        g_ind = 0
        g_vector = self.goals[g_ind] * self.gamma
        #Vector from position to goal
        t_vector = g_vector - p_vector
        
        #Rotates initial vector to face first target
        theta = calc_angle(m_vector, t_vector)
        m_vector = rotate_vec(m_vector, theta)

        #Only moves rat whils there are still goals to move to
        running = True

        while  running:
            g_ind, m_vector, p_vector, g_vector, vel_recordings, running = self.runStep(g_ind, m_vector, p_vector, g_vector, vel_recordings, running)
            self.system.DoStepDynamics(self.dt)
            
        self.reset()
        return vel_recordings

In [5]:
np.log10(40)

1.6020599913279623

In [9]:
goalsAlt = [[0,1],[1,1],[1,0],[2,0],[3,0],[3,1],[4,1],[4,2],[0,2]]

test = ratSim(theta_c=0.5, vec_mag=1)
velocity_data = pd.DataFrame(test.runIRR(), columns=['x','z'])

Goal reached, moving on
Goal reached, moving on
Goal reached, moving on
Goal reached, moving on
Goal reached, moving on


In [26]:
velocity_data.to_csv('velocity_data.csv')

In [24]:
velocity_data.shape

(1559, 2)

In [15]:
m.atan(1/-0.53382566)

-1.0804557795924132

In [18]:
np.linalg.norm(-10)

10.0

In [21]:
np.sign(5)

1

In [3]:
theta = m.pi/5
theta2 = m.pi + theta
a = np.asarray([m.cos(theta), m.sin(theta)])
print('a:', a)

b = np.asarray([-m.sin(theta), m.cos(theta)])
print('b:', b)

c = np.asarray([m.cos(theta2), m.sin(theta2)])
print('c:', c)

d = np.asarray([-m.sin(theta2), m.cos(theta2)])
print('d:', b)

a: [0.80901699 0.58778525]
b: [-0.58778525  0.80901699]
c: [-0.80901699 -0.58778525]
d: [-0.58778525  0.80901699]


In [3]:
print(m.atan(-0.9/-0.1))

1.460139105621001


In [4]:
print(m.atan(-0.9/0.1))

-1.460139105621001
